In [11]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from matplotlib import mlab
from scipy import signal, stats
from sklearn import preprocessing
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from datetime import datetime
from datetime import timedelta
import jpholiday
from plotly.subplots import make_subplots
import plotly.express as px

Tokyo_station_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Tokyostation.npy"
Tokyo_station_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Tokyostaion.npy"

Kyoto_station_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Kyotostation.npy"
Kyoto_station_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Kyotostation.npy"

Arashi_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Arashiyama.npy"
Arashi_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Arashiyama.npy"

Karasuma_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Karasuma.npy"
Karasuma_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Karasuma.npy"

Kinkaku_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Kinkaku.npy"
Kinkaku_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Kinkaku.npy"

Kiyomizu_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Kiyomizu.npy"
Kiyomizu_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Kiyomizu.npy"

University_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/University.npy"
University_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/University.npy"

Nizyou_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Nizyou.npy"
Nizyou_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Nizyou.npy"

Touzi_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Touzi.npy"
Touzi_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Touzi.npy"

list_mobile = [
    Tokyo_station_mobile_PATH,
    Arashi_mobile_PATH,
    Karasuma_mobile_PATH,
    Kinkaku_mobile_PATH,
    Kiyomizu_mobile_PATH,
    University_mobile_PATH,
    Nizyou_mobile_PATH,
    Touzi_mobile_PATH,
    Kyoto_station_mobile_PATH,
]
list_twitter = [
    Tokyo_station_tweets_PATH,
    Arashi_tweets_PATH,
    Karasuma_tweets_PATH,
    Kinkaku_tweets_PATH,
    Kiyomizu_tweets_PATH,
    University_tweets_PATH,
    Nizyou_tweets_PATH,
    Touzi_tweets_PATH,
    Kyoto_station_tweets_PATH,
]
list_key = [
    "(a) Tokyo Station",
    "(e) Arashiyama",
    "(c) Nakagyou Ward",
    "(f) Kinkaku-ji temple",
    "(g) Kiyomizu-dera temple",
    "(d) Kyoto University",
    "(h) Nijo-jo castle",
    "(i) To-ji temple",
    "(b) Kyoto Station",
]

In [12]:
def isBizDay(DATE):
    # DATE = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    if DATE.weekday() >= 5 or jpholiday.is_holiday(DATE):
        return "Holiday"
    else:
        return "Workday"

In [13]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df_work= pd.DataFrame(data,columns=columns)
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']
color=['red','orange','green','aquamarine','blue','aqua','purple','pink','lime']

df_holi= pd.DataFrame(data,columns=columns)



In [14]:
fig_workday = make_subplots(rows=1, cols=1, shared_yaxes='all', shared_xaxes='all')

list_place = df_work['place'].unique()

# Define colors
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22']

# Iterate through places and add traces to the first subplot
for idx, place in enumerate([list_place[0], list_place[8], list_place[2], list_place[5], list_place[1], list_place[3], list_place[4], list_place[6], list_place[7]]):
    df_place_work = df_work[df_work['place'] == place]
    fig_workday.add_trace(go.Scatter(x=df_place_work['time'], y=df_place_work['r'], name=place, line=dict(color=colors[idx])))

# Update layout and axis labels for 'Workday' subplot
fig_workday.update_layout(
    yaxis=dict(range=(-1, 1), showline=False, showgrid=False, linecolor='black', linewidth=1, tickvals=[-0.5, 0, 0.5], ticktext=['-0.5', '0', '0.5'], zeroline=False, tickmode='array', tickangle=0),
    xaxis_title='hour',
    yaxis_title='social sensor performance',
    paper_bgcolor='white',
    plot_bgcolor='white',
    width=700,
    height=500
)

fig_workday.update_yaxes(showgrid=True, gridcolor='black')
fig_workday.update_xaxes(linecolor='black', linewidth=1, mirror=True)

# Remove legend from 'Workday' subplot
fig_workday.update_traces(showlegend=False)

# Create the second subplot for 'Holiday'
fig_holiday = make_subplots(rows=1, cols=1, shared_yaxes='all', shared_xaxes='all')

# Iterate through places and add traces to the second subplot
for idx, place in enumerate([list_place[0], list_place[8], list_place[2], list_place[5], list_place[1], list_place[3], list_place[4], list_place[6], list_place[7]]):
    df_place_holi = df_holi[df_holi['place'] == place]
    fig_holiday.add_trace(go.Scatter(x=df_place_holi['time'], y=df_place_holi['r'], name=place, line=dict(color=colors[idx])))

# Update layout and axis labels for 'Holiday' subplot
fig_holiday.update_layout(
    yaxis=dict(range=(-1, 1), showline=False, showgrid=False, linecolor='black', linewidth=1, tickvals=[-0.5, 0, 0.5], ticktext=['-0.5', '0', '0.5'], zeroline=False, tickmode='array', tickangle=0),
    xaxis_title='hour',
    yaxis_title='social sensor performance',
    paper_bgcolor='white',
    plot_bgcolor='white',
    width=700,
    height=500
)

fig_holiday.update_yaxes(showgrid=True, gridcolor='black')
fig_holiday.update_xaxes(linecolor='black', linewidth=1, mirror=True)

# Remove legend from 'Holiday' subplot
fig_holiday.update_traces(showlegend=False)

# Show the 'Workday' plot
fig_workday.show()

# Show the 'Holiday' plot
fig_holiday.show()

# Create a separate legend figure
legend_fig = go.Figure()
for place, color in zip(list_place, colors):
    legend_fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color=color), showlegend=True, name=place))

# Configure the layout for the legend
legend_fig.update_layout(
    showlegend=True,
    paper_bgcolor='white',
    plot_bgcolor='white',
    width=700,
    height=500
  # Adjust the height to your preference
)

# Show the legend figure
legend_fig.show()

In [15]:
fig = make_subplots(rows=1, cols=2,subplot_titles=['Workdays','Holidays'], shared_yaxes='all', shared_xaxes='all')


list_place = df_work['place'].unique()

# Define colors
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22']

# Iterate through places and add traces to the first subplot
for idx, place in enumerate([list_place[0], list_place[8], list_place[2], list_place[5], list_place[1], list_place[3], list_place[4], list_place[6], list_place[7]]):
    df_place_work = df_work[df_work['place'] == place]
    fig.add_trace(go.Scatter(x=df_place_work['time'], y=df_place_work['r'], name=place, line=dict(color=colors[idx])),row=1, col=1)



# Create the second subplot for 'Holiday'

# Iterate through places and add traces to the second subplot
for idx, place in enumerate([list_place[0], list_place[8], list_place[2], list_place[5], list_place[1], list_place[3], list_place[4], list_place[6], list_place[7]]):
    df_place_holi = df_holi[df_holi['place'] == place]
    fig.add_trace(go.Scatter(x=df_place_holi['time'], y=df_place_holi['r'], name=place, line=dict(color=colors[idx]),showlegend=False),row=1, col=2)

# Update layout and axis labels for 'Holiday' subplot
fig.update_layout(
    yaxis=dict(range=(-1, 1), showline=False, showgrid=False, linecolor='black', linewidth=1, tickvals=[-0.5, 0, 0.5], ticktext=['-0.5', '0', '0.5'], zeroline=False, tickmode='array', tickangle=0),
    xaxis_title='hour',
    yaxis_title='social sensor performance',
    paper_bgcolor='white',
    plot_bgcolor='white',
    width=1200,
    height=500,
    )
fig.update_xaxes(title='hour', row=1, col=2)
fig.update_yaxes(
    dict(range=(-1, 1), showline=False, showgrid=False, linecolor='black', linewidth=1, tickvals=[-0.5, 0, 0.5], ticktext=['-0.5', '0', '0.5'], zeroline=False, tickmode='array', tickangle=0),
    row=1, col=2
    )


fig.update_yaxes(showgrid=True, gridcolor='black')
fig.update_xaxes(linecolor='black', linewidth=1, mirror=True)



# Show the 'Workday' plot
fig.show()
